In [4]:
import os
import pandas as pd
from datetime import datetime

In [11]:
os.listdir()

df = pd.DataFrame()

for csv_file in os.listdir():
    if 'csv' in csv_file:
        if csv_file == 'CARD_SUBWAY_MONTH_202005.csv':
            break
            
        else:
            df2 = pd.read_csv(csv_file, engine = "python")
            df = pd.concat([df, df2])

In [12]:
df

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20150101,2호선,낙성대,14586.0,14889.0,20151217
1,20150101,2호선,사당,19233.0,20298.0,20151217
2,20150101,2호선,방배,5920.0,6065.0,20151217
3,20150101,2호선,서초,4379.0,4120.0,20151217
4,20150101,분당선,선정릉,1972.0,1828.0,20151217
...,...,...,...,...,...,...
17717,20200430,7호선,상동,7730.0,7786.0,20200503
17718,20200430,7호선,삼산체육관,2616.0,2286.0,20200503
17719,20200430,7호선,굴포천,4990.0,5027.0,20200503
17720,20200430,7호선,부평구청,5413.0,4684.0,20200503


In [13]:
def date_changer(x):
    x = str(x)
    date = x[:4] + '-' + x[4:6]
    return date

df['사용일자'] = df['사용일자'].apply(date_changer)
df['datetime'] = pd.to_datetime(df['사용일자'])

df = df[['노선명', '승차총승객수', '하차총승객수', 'datetime']]
df

,노선명,승차총승객수,하차총승객수,datetime
0,2호선,14586.0,14889.0,2015-01-01
1,2호선,19233.0,20298.0,2015-01-01
2,2호선,5920.0,6065.0,2015-01-01
3,2호선,4379.0,4120.0,2015-01-01
4,분당선,1972.0,1828.0,2015-01-01
...,...,...,...,...
17717,7호선,7730.0,7786.0,2020-04-01
17718,7호선,2616.0,2286.0,2020-04-01
17719,7호선,4990.0,5027.0,2020-04-01
17720,7호선,5413.0,4684.0,2020-04-01


In [14]:
df['노선명'].unique()

array(['2호선', '분당선', '수인선', '경원선', '일산선', '5호선', '6호선', '7호선', '경부선',
       '1호선', '안산선', '8호선', '공항철도 1호선', '경인선', '경춘선', '경의선', '중앙선', '9호선',
       '과천선', '장항선', '4호선', '3호선', '9호선2단계', '경강선', '우이신설선', '9호선2~3단계'],
      dtype=object)

In [146]:
def subway(line):
    subway_on_off = df.loc[df['노선명'] == line]
    subway_on_off = subway_on_off.set_index('datetime')
    subway_on_off = subway_on_off[['승차총승객수', '하차총승객수']].resample('M').mean()
    subway_on_off.index = subway_on_off.index.strftime('%Y %m')
    subway_on_off['승차총승객수'] = subway_on_off['승차총승객수'].astype(int)
    subway_on_off['하차총승객수'] = subway_on_off['하차총승객수'].astype(int)
    subway_on_off = subway_on_off.T
    subway_on_off.reset_index(inplace=True)
    subway_on_off.insert(0, '호선', line)
    subway_on_off.rename(columns = {'index' : '승하차'}, inplace = True)
    return subway_on_off

In [153]:
last_df = pd.DataFrame()

for i in list(df['노선명'].unique()):
    last_df = pd.concat([last_df, subway(i)], ignore_index=True)
    
last_df.to_csv('final.csv')

In [154]:
last_df

,호선,승하차,2015 01,2015 02,2015 03,2015 04,2015 05,2015 06,2015 07,2015 08,...,2019 07,2019 08,2019 09,2019 10,2019 11,2019 12,2020 01,2020 02,2020 03,2020 04
0,2호선,승차총승객수,31052.0,29290.0,33050.0,33127.0,31430.0,28940.0,30549.0,29045.0,...,31096.0,29592.0,28813.0,31483.0,31619.0,31380.0,27726.0,24518.0,18590.0,20273.0
1,2호선,하차총승객수,31269.0,29467.0,33276.0,33372.0,31785.0,29299.0,30919.0,29361.0,...,31557.0,30048.0,29286.0,31981.0,32098.0,31874.0,28113.0,24855.0,18841.0,20551.0
2,분당선,승차총승객수,10237.0,9690.0,11206.0,11293.0,10843.0,9708.0,10306.0,9802.0,...,10919.0,10284.0,10380.0,11255.0,11309.0,11062.0,9954.0,8720.0,6385.0,7242.0
3,분당선,하차총승객수,10571.0,9990.0,11584.0,11695.0,11169.0,10095.0,10678.0,10117.0,...,11297.0,10630.0,10726.0,11622.0,11696.0,11419.0,10278.0,9043.0,6633.0,7522.0
4,수인선,승차총승객수,2737.0,2624.0,3087.0,3234.0,3301.0,2930.0,2938.0,2907.0,...,3453.0,3273.0,3499.0,3825.0,3668.0,3465.0,3016.0,2624.0,2114.0,2390.0
5,수인선,하차총승객수,2750.0,2634.0,3102.0,3245.0,3310.0,2943.0,2939.0,2904.0,...,3460.0,3289.0,3515.0,3855.0,3691.0,3469.0,3030.0,2630.0,2123.0,2402.0
6,경원선,승차총승객수,7084.0,6929.0,8188.0,8346.0,8246.0,7260.0,7314.0,7122.0,...,7259.0,7011.0,7441.0,7940.0,7725.0,7372.0,6516.0,5641.0,4510.0,4932.0
7,경원선,하차총승객수,6940.0,6787.0,7989.0,8144.0,8038.0,7097.0,7137.0,6958.0,...,7034.0,6792.0,7202.0,7668.0,7470.0,7119.0,6320.0,5477.0,4380.0,4810.0
8,일산선,승차총승객수,9747.0,9320.0,10693.0,11326.0,10941.0,9560.0,10077.0,9814.0,...,11369.0,11051.0,11087.0,11958.0,11998.0,11606.0,10463.0,8956.0,7037.0,7726.0
9,일산선,하차총승객수,9431.0,9037.0,10316.0,10917.0,10559.0,9211.0,9741.0,9515.0,...,10983.0,10674.0,10673.0,11509.0,11521.0,11115.0,10095.0,8650.0,6801.0,7434.0
